In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import mr

In [3]:
from mrcnn import model as modellib
from mrcnn import visualize
from mrcnn import utils

Using TensorFlow backend.


In [4]:
os.chdir(os.path.join("E:",os.sep,"RCNNTanks512Train"))

In [5]:
ds_valid = bp.BPDataset()
ds_valid.load("valid.json")
ds_valid.prepare()

In [6]:
inference_config = bp.InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir="logs")

model_path = model.find_last()
print("Loading weights from ", model_path)
# model_path = "mask_rcnn_bp_0500.h5"
# model.load_weights(model_path, by_name=True)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Loading weights from  logs\bp20190409T0949\mask_rcnn_bp_0500.h5


In [7]:
dataset = ds_valid
image_ids = dataset.image_ids
APs = []
for image_id in image_ids[:30]:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

mAP:  0.0
